In [1]:
import torch

from models import S2SPL
from dataset import en_de_dataset
from utils import load_config_json
import os
from torchtext.data.metrics import bleu_score

C:\Users\83577\anaconda3\envs\torch\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
C:\Users\83577\anaconda3\envs\torch\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(


In [2]:
path = os.listdir('best')[-1] # could be replaced by the folder name
path = os.path.join('best', path)

checkpoint = os.listdir(path)[0] # could be replaced by your own
checkpoint = os.path.join(path, checkpoint)

params = os.path.join(path, 'config.json')

print(path + '\n')
print(checkpoint + '\n')
print(params)

# Initialize args
args = load_config_json(params)

args['DEVICE'] = 'cpu'

best\DEVICEcuda BATCH_SIZE256 LEARNING_RATE0.0001 EPOCHS300 ENCODER_TYPEGRU DECODER_TYPEGRU ATTENTION_HEAD1 ENCODER_LAYERS2 DECODER_LAYERS4 EMB_DIM300 HIDDEN_ENCODER1024 HIDDEN_DECODER1024 DROPOUT_ENCODER0.5 DROPOUT_DECODER0.5 BIDIRECTIONTrue PAIR('de', 'en')

best\DEVICEcuda BATCH_SIZE256 LEARNING_RATE0.0001 EPOCHS300 ENCODER_TYPEGRU DECODER_TYPEGRU ATTENTION_HEAD1 ENCODER_LAYERS2 DECODER_LAYERS4 EMB_DIM300 HIDDEN_ENCODER1024 HIDDEN_DECODER1024 DROPOUT_ENCODER0.5 DROPOUT_DECODER0.5 BIDIRECTIONTrue PAIR('de', 'en')\-epoch=28-valid_loss=2.99-validation.ckpt

best\DEVICEcuda BATCH_SIZE256 LEARNING_RATE0.0001 EPOCHS300 ENCODER_TYPEGRU DECODER_TYPEGRU ATTENTION_HEAD1 ENCODER_LAYERS2 DECODER_LAYERS4 EMB_DIM300 HIDDEN_ENCODER1024 HIDDEN_DECODER1024 DROPOUT_ENCODER0.5 DROPOUT_DECODER0.5 BIDIRECTIONTrue PAIR('de', 'en')\config.json


In [3]:
# Initial Dataset
trainset = en_de_dataset(split='train', pair=args['PAIR'])
# Initialize model parameters
s2s_model = S2SPL(
        vocab_size_encoder=len(trainset.input_vocab),
        vocab_size_decoder=len(trainset.target_vocab),
        input_vocab=trainset.input_vocab,
        target_vocab=trainset.target_vocab,
        input_id_to_word=trainset.id_to_word_input,
        target_id_to_word=trainset.id_to_word_target,
        args=args
)


model = s2s_model.load_from_checkpoint(checkpoint)
model.eval()

Input vocab size: 8014
Target vocab size: 6191


S2SPL(
  (encoder): RNNEncoder(
    (emb): Embedding(8014, 300)
    (rnn): GRU(300, 1024, num_layers=2, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (output_reduce): Linear(in_features=2048, out_features=1024, bias=True)
  )
  (decoder): RNNDecoder(
    (emb): Embedding(6191, 300)
    (rnn): GRU(1324, 1024, num_layers=4, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
    (softmax): Softmax(dim=2)
    (decode): Linear(in_features=1024, out_features=6191, bias=True)
  )
  (cross_entrophy): CrossEntropyLoss()
)

In [4]:
output = model.translate('Ein Mann geht am Strasse', 'A man walking on the street.' )

output

('A man is on the the of the <unk> .', 0.0)

In [20]:
input_pair, target_pair = trainset.random_pairs()
print((input_pair, target_pair))
print(model.translate(input_pair, target_pair))

('Fünf Männer in roten Oberteilen stehen auf einem Basketballfeld in einer Reihe, vier von ihnen haben die Hand auf dem Herzen und schauen nach oben.', 'Five men in red shirts, in a line, on a basketball court, four with hands on their hearts, and same four looking upward.')
('Five men in white uniforms are standing on a field , one of a , one of the , and', 0.0)


In [6]:
# import math
# def attn(query, key, value, input_indice):
#
#         input_indice_ext = input_indice.unsqueeze(1).repeat(1, query.shape[0], 1)
#         query = query.permute(1, 0, 2) # [seq_dec, batch, features] -> [batch, seq_dec, features]
#         key = key.permute(1, 2, 0) # [seq_enc, batch, features] -> [batch, features, seq_enc]
#         value =  value.permute(1, 0, 2)
#         # Getting attention score with mask
#         score = torch.bmm(query, key)/ math.sqrt(query.shape[1]) # [batch, seq_dec, seq_enc]
#         score[input_indice_ext == 0] = -9e3
#
#
#         score = torch.softmax(score, dim=2)
#         print(score)
#
#         attn_output = torch.bmm(score, value)
#         print(attn_output.shape)
#         return attn_output.permute(1, 0, 2)
#
#
#
# query = torch.randn(2, 2, 300)
#
# input_indice = torch.randint(4, 9000, (2, 10))
# input_indice[0, -4:] = 0
# input_indice[1, -6:] = 0
# input_indice[0, -4] = 3
# input_indice[1, -6] = 3
# key = torch.randn(10, 2, 300)
# value = key
#
# print(attn(query, key, value, input_indice).shape)
# print(key[(input_indice == 3).T].unsqueeze(0).shape, key[-1].unsqueeze(0).shape)

tensor([[[2.6760e-10, 2.5472e-12, 5.6875e-14, 1.2963e-06, 6.2832e-12,
          9.7142e-07, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [9.7744e-01, 1.2379e-12, 2.2534e-02, 8.1934e-07, 1.4084e-16,
          2.9267e-05, 5.4172e-09, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[9.2791e-03, 2.6948e-13, 7.8816e-06, 9.8889e-01, 1.8275e-03,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [9.9858e-01, 5.6932e-12, 1.7519e-17, 2.5815e-05, 1.3911e-03,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]]])
torch.Size([2, 2, 300])
torch.Size([2, 2, 300])
torch.Size([1, 2, 300]) torch.Size([1, 2, 300])
